<center><h2>Generating Transcript </h2>
<h4>ABC format Final</h4>
</center>

**Table of content**:

>1. It accepts csv with all the courses of a student in a multiple rows (ABC format)
>2. Splits the row into different rows resulting into a dataset of course and marks
>3.  


## 1. Importing Libraries and Reading Files 

In [2]:
from reportlab.pdfgen import canvas
import pandas as pd
from datetime import datetime
import warnings
import os
import re
warnings.filterwarnings("ignore")

In [3]:
from reportlab.platypus import (
    BaseDocTemplate, PageTemplate, Frame, Table, TableStyle,
    Spacer, Paragraph, NextPageTemplate
)

In [4]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.platypus import KeepInFrame, SimpleDocTemplate, Table, TableStyle, Spacer, Paragraph, \
                               KeepTogether, Frame, PageTemplate, PageBreak, Flowable

from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_LEFT

In [5]:
from xml.sax.saxutils import escape


In [7]:
styles = getSampleStyleSheet()

In [9]:
# University logo
IMAGE_PATH = r"D:\data\gcu\images\GCU Logo transparent.png"

# Base directories
base_dir   = r"D:\data\gcu\erp\transcript"
PHOTO_DIR  = os.path.join(base_dir, "photos")          # if you store photos here
output_dir = os.path.join(base_dir, "transcripts_pdf") # output folder for PDFs

# Ensure output folder exists
os.makedirs(output_dir, exist_ok=True)

In [11]:
df = pd.read_excel(os.path.join(base_dir, "data.xlsx")) 
df.head(1)

,ORG_NAME,ORG_CITY,ORG_STATE,ORG_PIN,ACADEMIC_COURSE_ID,COURSE_NAME,ADMISSION_YEAR,DEPARTMENT,STREAM,SESSION,...,SUB8_GRADE,SUB8_GRADE_POINTS,SUB8_CREDIT,SUB8_CREDIT_POINTS,SUB9NM,SUB9,SUB9_GRADE,SUB9_GRADE_POINTS,SUB9_CREDIT,SUB9_CREDIT_POINTS
0,Girijananda Chowdhury University,Guwahati,Assam,781017,2015,Master of Commerce (M.Com),2023,Commerce,NaN,2023-2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Strip spaces from column names
df.columns = df.columns.str.strip()
df.columns

Index(['ORG_NAME', 'ORG_CITY', 'ORG_STATE', 'ORG_PIN', 'ACADEMIC_COURSE_ID',
       'COURSE_NAME', 'ADMISSION_YEAR', 'DEPARTMENT', 'STREAM', 'SESSION',
       'ABC_ACCOUNT_ID', 'DOB', 'GENDER', 'CNAME', 'FNAME', 'MNAME',
       'MRKS_REC_STATUS', 'YEAR', 'MONTH', 'SEM', 'RROLL', 'REGN_NO',
       'TOT_GRADE', 'TOT_CREDIT', 'TOT_CREDIT_POINTS', 'TOT_GRADE_POINTS',
       'PERCENT', 'CGPA', 'SGPA', 'NCRF_LEVEL', 'GRAND_TOT_CREDIT_POINTS',
       'GRAND_TOT_CREDIT', 'RESULT', 'REMARKS', 'DOI', 'SUB1NM', 'SUB1',
       'SUB1_GRADE', 'SUB1_GRADE_POINTS', 'SUB1_CREDIT', 'SUB1_CREDIT_POINTS',
       'SUB2NM', 'SUB2', 'SUB2_GRADE', 'SUB2_GRADE_POINTS', 'SUB2_CREDIT',
       'SUB2_CREDIT_POINTS', 'SUB3NM', 'SUB3', 'SUB3_GRADE',
       'SUB3_GRADE_POINTS', 'SUB3_CREDIT', 'SUB3_CREDIT_POINTS', 'SUB4NM',
       'SUB4', 'SUB4_GRADE', 'SUB4_GRADE_POINTS', 'SUB4_CREDIT',
       'SUB4_CREDIT_POINTS', 'SUB5NM', 'SUB5', 'SUB5_GRADE',
       'SUB5_GRADE_POINTS', 'SUB5_CREDIT', 'SUB5_CREDIT_POINTS', 'SUB6

In [13]:
df.shape

(62, 89)

In [15]:
#data["COURSE_NAME"] = data["COURSE_NAME"].str.replace(r"\s+", " ", regex=True).str.strip()
#data["DEPARTMENT"] = data["DEPARTMENT"].str.replace(r"\s+", " ", regex=True).str.strip()

In [16]:
# Select all columns matching SUBxNM pattern
"""
sub_nm_cols = [col for col in df.columns if col.startswith("SUB") and col.endswith("NM")]

# Clean each column: remove newlines, tabs, extra spaces
for col in sub_nm_cols:
    df[col] = (
        df[col]
        .astype(str)                                  # make sure values are strings
        .str.replace(r"\s+", " ", regex=True)         # replace multiple whitespace (including \n, \t) with single space
        .str.strip()                                  # trim leading/trailing spaces
        .replace("nan", pd.NA)                        # convert "nan" back to missing
    )
"""

'\nsub_nm_cols = [col for col in df.columns if col.startswith("SUB") and col.endswith("NM")]\n\n# Clean each column: remove newlines, tabs, extra spaces\nfor col in sub_nm_cols:\n    df[col] = (\n        df[col]\n        .astype(str)                                  # make sure values are strings\n        .str.replace(r"\\s+", " ", regex=True)         # replace multiple whitespace (including \n, \t) with single space\n        .str.strip()                                  # trim leading/trailing spaces\n        .replace("nan", pd.NA)                        # convert "nan" back to missing\n    )\n'

In [17]:
# i am trying this

### 2. Convert rows (student with multiple subjects) into multiple rows for a single student

In [19]:
def expand_student_rows(df):
    """
    Converts each row (student with multiple subjects) into multiple rows 
    (one row per subject).
    """

    # Identify how many subjects are in the data (e.g., SUB1, SUB2, ...)
    subject_nums = sorted({col[3] for col in df.columns if col.startswith("SUB") and col[3].isdigit()})

    all_rows = []

    for _, row in df.iterrows():
        for num in subject_nums:
            # Build one new row for each subject
            new_row = {
                # Copy all the student-level info that stays the same
                "ORG_NAME": row["ORG_NAME"],
                "ORG_PIN": row["ORG_PIN"],
                "ACADEMIC_COURSE_ID": row["ACADEMIC_COURSE_ID"],
                "COURSE_NAME": row["COURSE_NAME"],
                "ADMISSION_YEAR": row["ADMISSION_YEAR"],
                "DEPARTMENT": row["DEPARTMENT"],
                "STREAM": row["STREAM"],
                "SESSION": row["SESSION"],
                "ABC_ACCOUNT_ID": row["ABC_ACCOUNT_ID"],
                "DOB": row["DOB"],
                "GENDER": row["GENDER"],
                "MRKS_REC_STATUS": row["MRKS_REC_STATUS"],
                "CNAME": row["CNAME"],
                "YEAR": row["YEAR"],
                "MONTH": row["MONTH"],
                "SEM": row["SEM"],
                "RROLL": row["RROLL"],
                "REGN_NO": row["REGN_NO"],
                "TOT_GRADE": row["TOT_GRADE"],
                "TOT_CREDIT": row["TOT_CREDIT"],
                "TOT_CREDIT_POINTS": row["TOT_CREDIT_POINTS"],
                "TOT_GRADE_POINTS": row["TOT_GRADE_POINTS"],
                "PERCENT": row["PERCENT"],
                "CGPA": row["CGPA"],
                "SGPA": row["SGPA"],
                "GRAND_TOT_CREDIT_POINTS": row["GRAND_TOT_CREDIT_POINTS"],
                "GRAND_TOT_CREDIT": row["GRAND_TOT_CREDIT"],
                "RESULT": row["RESULT"],
                "REMARKS": row["REMARKS"],
                "DOI": row["DOI"],
                "NCRF_LEVEL": row["NCRF_LEVEL"],

                # Subject-specific columns
                "SUB_NAME": row[f"SUB{num}NM"],
                "SUB_CODE": row[f"SUB{num}"],
                "GRADE": row[f"SUB{num}_GRADE"],
                "GRADE_POINTS": row[f"SUB{num}_GRADE_POINTS"],
                "CREDIT": row[f"SUB{num}_CREDIT"],
                "CREDIT_POINTS": row[f"SUB{num}_CREDIT_POINTS"]
            }
            all_rows.append(new_row)

    return pd.DataFrame(all_rows)


In [20]:
df_processed = expand_student_rows(df)
#df_processed

In [22]:
#serial_no = REGN_NO

#### The following is just a test to each student's semester 1 result 
this should be avoided for general report generation

In [23]:

rows = []

#for (student_id, student_data) in df_processed.groupby("RROLL"):
#    generate_pdf(student_id, student_data)

for (student_id, student_data) in df_processed.groupby("RROLL"):
    # keep only Semester 1 for this student
    sem1_data = student_data[student_data["SEM"] == 'I']

    # if there are multiple rows in sem1_data, we just take the first row
    # (since the columns you're interested in are per-student summary fields)
    row = sem1_data.iloc[0]

    rows.append({
        
        "SERIAL_NO": row["REGN_NO"],
        "REGN_NO": student_id,
        #"SEM": row["SEM"],
        #"NAME":row[""]
        "TOT_CREDIT_POINTS": row["TOT_CREDIT_POINTS"],
        "TOT_GRADE_POINTS": row["TOT_GRADE_POINTS"],
        "PERCENT": row["PERCENT"],
        "CGPA": row["CGPA"],
        "SGPA": row["SGPA"],
        "GRAND_TOT_CREDIT_POINTS": row["GRAND_TOT_CREDIT_POINTS"],
        "GRAND_TOT_CREDIT": row["GRAND_TOT_CREDIT"],
        "RESULT": row["RESULT"],
        "COURSE_NAME": row["COURSE_NAME"]
    })

# convert to a dataframe
summary_df = pd.DataFrame(rows)


In [24]:
summary_df.head()

,SERIAL_NO,REGN_NO,TOT_CREDIT_POINTS,TOT_GRADE_POINTS,PERCENT,CGPA,SGPA,GRAND_TOT_CREDIT_POINTS,GRAND_TOT_CREDIT,RESULT,COURSE_NAME
0,2025100002,2312004003,173,NaN,85.1,7.86,7.86,173.0,22.0,Cleared,M.Sc in Botany
1,2025100003,2312004006,204,NaN,91.4,9.27,9.27,204.0,22.0,Cleared,M.Sc in Botany
2,2025100004,2312005001,173,NaN,85.8,7.86,7.86,173.0,22.0,Cleared,M.Tech in Thermal and Fluid Engineering
3,2025100005,2312005003,170,NaN,84.9,7.73,7.73,170.0,22.0,Cleared,M.Tech in Thermal and Fluid Engineering
4,2025100006,2312010001,187,NaN,85.2,8.13,8.13,187.0,23.0,Cleared,MA in English


In [25]:
df_processed.columns

Index(['ORG_NAME', 'ORG_PIN', 'ACADEMIC_COURSE_ID', 'COURSE_NAME',
       'ADMISSION_YEAR', 'DEPARTMENT', 'STREAM', 'SESSION', 'ABC_ACCOUNT_ID',
       'DOB', 'GENDER', 'MRKS_REC_STATUS', 'CNAME', 'YEAR', 'MONTH', 'SEM',
       'RROLL', 'REGN_NO', 'TOT_GRADE', 'TOT_CREDIT', 'TOT_CREDIT_POINTS',
       'TOT_GRADE_POINTS', 'PERCENT', 'CGPA', 'SGPA',
       'GRAND_TOT_CREDIT_POINTS', 'GRAND_TOT_CREDIT', 'RESULT', 'REMARKS',
       'DOI', 'NCRF_LEVEL', 'SUB_NAME', 'SUB_CODE', 'GRADE', 'GRADE_POINTS',
       'CREDIT', 'CREDIT_POINTS'],
      dtype='object')

In [26]:
if "NCRF_LEVEL" not in df_processed.columns:  
    df_processed["NCRF_LEVEL"] = ""


df_processed["NCRF_LEVEL"] = df_processed["NCRF_LEVEL"].fillna(" ")
df_processed = df_processed.dropna(subset=['SUB_NAME', 'SUB_CODE'], how='all')

In [27]:
#one_student.info()

#### Just trying to print a students semester results

In [28]:
# Sunit dey
#one_student = df_processed[['RROLL','COURSE_NAME','STREAM','CNAME','SEM','SUB_NAME','SUB_CODE', 'GRADE']]
#one_student = one_student.dropna(subset=['SUB_NAME', 'SUB_CODE'], how='all')
#one_student_filtered = one_student[(one_student['RROLL'].astype(str) == "2312004006") & (one_student['SEM'].str.strip() == "I")]
#one_student_filtered

In [31]:
# ---------------- SAFE HELPERS ----------------
def safe_round(value, ndigits=2, default=0):
    try:
        return round(float(value), ndigits)
    except (ValueError, TypeError):
        return default

def remove_decimal(value):
    try:
        val = float(value)
        if val.is_integer():
            return int(val)
        return round(val, 2)
    except Exception:
        return value
        
# --- Safe paragraph wrapper ---
def safe_paragraph(text, style):
    """Wrap text safely in a Paragraph, escaping XML chars"""
    if text is None:
        text = ""
    return Paragraph(escape(str(text)), style)

# New helper function to clean up problematic characters
def clean_text_for_reportlab(text):
    if text is None:
        return ""
    # Convert to string and replace common problematic characters (e.g., non-breaking spaces)
    text = str(text).replace('\xa0', ' ').replace('\u2013', '-').strip()
    # If the issue is with special UTF-8 characters, you might need a more aggressive filter:
    # return text.encode('ascii', 'ignore').decode('ascii')
    
    return text

def safe_text(val, default=""):
    """Ensure text is safe for ReportLab Paragraphs/drawString."""
    if pd.isna(val) or val is None:
        return default
    return str(val)

def safe_number_text(val, default=""):
    """Ensure numbers don’t come in scientific notation."""
    try:
        return str(int(float(val)))
    except Exception:
        return default

In [32]:
def find_photo(photo_dir, enrollment_no):
    """
    Find the student's photo file based on enrollment number.
    Supports .jpg, .jpeg, and .png extensions.
    
    Returns:
        str | None: Full path to photo if found, else None.
    """
    extensions = ["jpg", "jpeg", "png"]
    for ext in extensions:
        candidate = os.path.join(photo_dir, f"{enrollment_no}.{ext}")
        if os.path.exists(candidate):
            return candidate
    return None


## 3. Draw Header

In [34]:
def draw_header_with_photo(canvas, doc, student_data):
    width, height = A4
    usable_width = width - doc.leftMargin - doc.rightMargin

    enrollment_no = safe_text(student_data.iloc[0].get("RROLL", ""))
    student_name = safe_text(student_data.iloc[0].get("CNAME", ""))
    apaar_id = safe_number_text(student_data.iloc[0].get("ABC_ACCOUNT_ID", ""))
    serial_no = safe_number_text(student_data.iloc[0].get("REGN_NO", ""))
    course_name = safe_text(student_data.iloc[0].get("COURSE_NAME", ""))
    level = safe_text(student_data.iloc[0].get("NCRF_LEVEL", ""))

    # --- Logo ---
    try:
        canvas.drawImage(IMAGE_PATH, 50, height - 140, width=80, height=80, mask='auto')
    except Exception:
        pass

    # --- Photo ---
    photo_file = find_photo(PHOTO_DIR, enrollment_no)

    if not photo_file:
        print(f"⚠️ No photo found for {student_name} ({enrollment_no})")
    try:
        if photo_file:
            canvas.drawImage(photo_file, 450, height - 140, width=70, height=80, mask='auto')
    except Exception:
        pass

    # --- Titles ---
    canvas.setFont("Times-Roman", 18)
    canvas.drawCentredString(width / 2, height - 60, "Girijananda Chowdhury University")
    canvas.setFont("Times-Roman", 14)
    canvas.drawCentredString(width / 2, height - 85, "TRANSCRIPT")
    canvas.setFont("Times-Roman", 12)
    canvas.drawCentredString(width / 2, height - 100, course_name)
    canvas.drawCentredString(width / 2, height - 115, "2023-2025")

    # --- Student data table ---
    std_data = [
        ["APAAR ID", ":", apaar_id, "Serial Number : " + serial_no],
        ["Enrollment No.", ":", enrollment_no, ""],
        ["Name", ":", student_name, ""],
        ["NCrF/NHEQF Level", ":", level, ""],
    ]
    std_table = Table(std_data, colWidths=[120, 10, 200, 150])
    std_table.setStyle(TableStyle([
        ("FONTNAME", (0,0), (-1,-1), "Helvetica"),
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("ALIGN", (0,0), (0,-1), "LEFT"),
        ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
        ("TOPPADDING", (0,0), (-1,-1), 1),
        ("BOTTOMPADDING", (0,0), (-1,-1), 1),
    ]))
    table_width, table_height = std_table.wrap(0, 0)
    std_table.drawOn(canvas, 50, height - 150 - table_height)


### 3.1 Styles

In [35]:
left_sem_style = ParagraphStyle(
    name="LeftSemesterHeading",
    parent=styles['Heading4'],
    alignment=TA_LEFT,
    leftIndent=-30,   # moves 15 points left of the normal margin
    firstLineIndent=0,
    fontName="Helvetica",   # <--- this removes italics
)

In [36]:
def format_subject_name(text, max_len=45):
    """
    Returns a Paragraph with adaptive font size.
    If the subject name is too long, reduce font size.
    """
    styles = getSampleStyleSheet()
    subject_style = styles["Normal"]
    subject_style.fontName = "Times-Roman"
    subject_style.leading = 11
    subject_style.alignment = TA_LEFT

    if len(str(text)) > max_len:  
        subject_style.fontSize = 8   # shrink if too long
        subject_style.leading = 10
    elif len(str(text)) > max_len * 1.5:  
        subject_style.fontSize = 7   # extra shrink
        subject_style.leading = 9
    else:
        subject_style.fontSize = 9   # normal case
    safe_text = escape(str(text))  # prevent invalid XML
    return Paragraph(str(text), subject_style)

# --- Safe paragraph wrapper ---
def safe_paragraph(text, style):
    """Wrap text safely in a Paragraph, escaping XML chars"""
    if text is None:
        text = ""
    return Paragraph(escape(str(text)), style)

def safe_number(x):
    """Ensure numeric values are safe for ReportLab (no NaN)."""
    try:
        if pd.isna(x):
            return 0
        return int(x) if float(x).is_integer() else round(float(x), 2)
    except Exception:
        return 0

## 4. Generate PDFs

In [37]:
def generate_pdf(student_id, student_data, report_date):
    width, height = A4
    #report_date = '25-09-2025'
    filename = os.path.join(output_dir, f"Transcript_{student_data.iloc[0]['CNAME']}.pdf")
    doc = BaseDocTemplate(filename, pagesize=A4)
    doc.showFooter = True

    # --- Styles ---
    styles = getSampleStyleSheet()
    subject_style = styles["Normal"]
    subject_style.fontName = "Times-Roman"
    subject_style.fontSize = 9
    subject_style.leading = 11
    subject_style.alignment = TA_LEFT

    bold_subject_style = ParagraphStyle(
        name="SubjectBold",
        parent=subject_style,
        fontName="Times-Bold",
        fontSize=subject_style.fontSize,
        leading=subject_style.leading,
        alignment=subject_style.alignment
    )

    left_sem_style = ParagraphStyle(
        name="LeftSemesterHeading",
        parent=styles['Heading4'],
        alignment=TA_LEFT,
        leftIndent=-30,
        fontName="Helvetica"
    )

    # --- Frames ---
    frame_first = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height - 140, id='first_frame')
    frame_later = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height + 50, id='later_frame')


    # --- Page templates ---
    first_page_template = PageTemplate(
        id='FirstPage',
        frames=[frame_first],
        onPage=lambda c, d: draw_header_with_photo(c, d, student_data)
        # Notice: no onPageEnd here, so no footer/signature on Page 1
    )
    
    middle_page_template = PageTemplate(
        id='MiddlePages',
        frames=[frame_later],
        onPageEnd=lambda c, d: draw_footer(c, d, report_date)
    )
    
    doc.addPageTemplates([first_page_template, middle_page_template])

    # --- Table style ---
    table_style = TableStyle([
        ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
        ("FONTNAME", (0,0), (-1,-1), "Times-Roman"),
        ("FONTSIZE", (0,0), (-1,-1), 9),
        ("FONTNAME", (0,0), (-1,0), "Times-Bold"),
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ("ALIGN", (2,1), (-1,-1), "CENTER"),
    ])

    elements = []

    # --- Group by SEM ---
    for i, (sem, sem_data) in enumerate(student_data.groupby("SEM")):
        if i == 1:
            elements.append(NextPageTemplate('MiddlePages'))

        # Build table rows
        data = [["Sub Code", "Subject/Papers", "Credit", "Grade", "Grade Points", "Credit Points"]]

        for _, row in sem_data.iterrows():
            if row[["SUB_CODE","SUB_NAME","CREDIT","GRADE","GRADE_POINTS","CREDIT_POINTS"]].isna().all():
                continue

            # Clean + escape subject code and subject name
            sub_code = str(row.get("SUB_CODE", "")).replace("/", "_")
            sub_name = format_subject_name(row.get("SUB_NAME", ""))

            data.append([
                escape(sub_code),
                sub_name,
                safe_number(row.get("CREDIT", 0)),
                escape(str(row.get("GRADE", ""))),
                safe_number(row.get("GRADE_POINTS", 0)),
                safe_number(row.get("CREDIT_POINTS", 0))
            ])

        if len(data) == 1:  # skip empty semesters
            continue

        # SGPA row
        total_credits = safe_number(sem_data["CREDIT"].sum())
        total_credit_points = safe_number(sem_data["CREDIT_POINTS"].sum())
        sgpa = round(total_credit_points / total_credits, 2) if total_credits else 0

        total_row = [
            Paragraph("Total", bold_subject_style),
            Paragraph(f"SGPA : {sgpa}", bold_subject_style),
            safe_number(total_credits),
            "",
            "",
            safe_number(total_credit_points)
        ]
        data.append(total_row)

        colWidths = [65, 220, 45, 45, 70, 65]
        heading = Paragraph(f"<b>Semester {sem}</b>", left_sem_style)

        table = Table(data, colWidths=colWidths, repeatRows=1)
        table.setStyle(table_style)

        elements.append(KeepTogether([heading, Spacer(1, 6), table]))
        elements.append(Spacer(1, 12))

    # --- Summary section ---
    try:
        cgpa = round(float(student_data["CGPA"].iloc[-1]), 2)
    except Exception:
        cgpa = 0.0

    percentage = round(cgpa * 10, 2)
    total_credit_points = safe_number(student_data["CREDIT_POINTS"].sum())
    total_credit = safe_number(student_data["CREDIT"].sum())
    result = escape(str(student_data.iloc[0].get("RESULT", "")))

    summary_data = [
        ["RESULT", f": {result}", "", "Grand Total Credit Points", f": {total_credit_points}"],
        ["CGPA", f": {cgpa}", "", "Grand Total Credit", f": {total_credit}"],
        ["Percentage", f": {percentage}", "", "", ""]
    ]

    usable_width = width - doc.leftMargin - doc.rightMargin
    colWidths = [0.12 * usable_width, 0.12 * usable_width, 0.30 * usable_width,
                 0.28 * usable_width, 0.28 * usable_width]

    summary_table = Table(summary_data, colWidths=colWidths, hAlign='LEFT')
    summary_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (1, 0), (1, -1), 'Helvetica-Bold'),
        ('FONTNAME', (4, 0), (4, -1), 'Helvetica-Bold'),
    ]))

    elements.append(summary_table)
    elements.append(Spacer(1, 20))

    # --- Build ---
    elements = [x for x in elements if x not in [None, "", []]]
    try:
        doc.build(elements, canvasmaker=NumberedCanvas)
        print(f"✅ PDF generated: {filename}")
    except Exception as e:
        print(f"❌ PDF build failed: {e}")


In [40]:
def remove_decimal(value):
    try:
        val = float(value)
        if val.is_integer():
            return int(val)
        return round(val, 2)
    except Exception:
        return value

def ensure_row(*cells):
    return list(cells)



In [41]:
def draw_footer(canvas, doc, date_value):
    # Skip footer on first page (page numbers are still handled by NumberedCanvas)
    #if doc.page == 1:
    #    return
    
    width, height = A4
    canvas.setFont("Helvetica", 10)
    canvas.drawString(50, 60, f"Date : {date_value}")
    canvas.drawRightString(width - 55, 70, "Controller of Examination")
    canvas.drawRightString(width - 40, 60, "Girijananda Chowdhury University")


In [42]:
def draw_first_page(canvas, doc):
    canvas.saveState()
    canvas.setFont("Times-Bold", 12)
    canvas.drawCentredString(doc.width/2.0 + doc.leftMargin, doc.height + doc.topMargin - 20, "Your Header for First Page")
    canvas.restoreState()

def draw_later_pages(canvas, doc):
    canvas.saveState()
    canvas.setFont("Times-Roman", 9)
    canvas.drawRightString(doc.width + doc.leftMargin, 20, f"Page {doc.page}")
    canvas.restoreState()


In [43]:
class SetFooterFlag(Flowable):
    """
    Flowable used to indicate that the footer should start appearing
    from this point onward.
    """
    def __init__(self, value=True):
        super().__init__()
        self.value = value

    def draw(self):
        # Store the flag on the doctemplate.
        # draw_footer() will read this flag.
        self._doctemplate.showFooter = self.value

In [44]:
class NumberedCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        super(NumberedCanvas, self).__init__(*args, **kwargs)
        self._saved_page_states = []

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """Add total page count to each page."""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_number(num_pages)
            super(NumberedCanvas, self).showPage()
        super(NumberedCanvas, self).save()

    def draw_page_number(self, page_count):
        page = f"Page {self._pageNumber} of {page_count}"
        self.setFont("Helvetica", 9)
        width, height = A4
        #self.drawRightString(width - 40, 30, page)  # bottom-right corner
        self.drawCentredString(width / 2.0, 30, page)  # bottom-center

In [45]:
def generate_pdf_onepage(student_id, student_data, report_date):
    """
    Generate a single-page transcript (no page numbers, but with footer).
    """
    width, height = A4
    #report_date ="27-09-2025"
    filename = os.path.join(output_dir, f"Transcript_{student_data.iloc[0]['CNAME']}_onepage.pdf")
    doc = BaseDocTemplate(filename, pagesize=A4)

    # --- Styles ---
    styles = getSampleStyleSheet()
    subject_style = styles["Normal"]
    subject_style.fontName = "Times-Roman"
    subject_style.fontSize = 9
    subject_style.leading = 11
    subject_style.alignment = TA_LEFT

    bold_subject_style = ParagraphStyle(
        name="SubjectBold",
        parent=subject_style,
        fontName="Times-Bold",
        fontSize=subject_style.fontSize,
        leading=subject_style.leading,
        alignment=subject_style.alignment
    )

    left_sem_style = ParagraphStyle(
        name="LeftSemesterHeading",
        parent=styles['Heading4'],
        alignment=TA_LEFT,
        leftIndent=-30,
        fontName="Helvetica"
    )

    # --- Frames ---
    frame_main = Frame(
        doc.leftMargin, doc.bottomMargin,
        doc.width, doc.height - 140,
        id="normal_frame"
    )
    

    # --- Page template (only one page, with footer) ---
    single_page_template = PageTemplate(
        id="OnePage",
        frames=[frame_main],
        onPage=lambda c, d: draw_header_with_photo(c, d, student_data),
        onPageEnd=lambda c, d: draw_footer(c, d, report_date)  # always show footer
    )
    doc.addPageTemplates([single_page_template])

    # --- Elements ---
    elements = []

    # Just one semester (or summary) — adapt depending on your case
    for sem, sem_data in student_data.groupby("SEM"):
        # Table header
        data = [["Sub Code", "Subject/Papers", "Credit",
                 "Grade", "Grade Points", "Credit Points"]]

        for _, row in sem_data.iterrows():
            if row[["SUB_CODE", "SUB_NAME", "CREDIT", "GRADE",
                    "GRADE_POINTS", "CREDIT_POINTS"]].isna().all():
                continue

            data.append([
                row.get("SUB_CODE", ""),
                safe_paragraph(row.get("SUB_NAME", ""), subject_style),
                row.get("CREDIT", 0) or 0,
                row.get("GRADE", ""),
                row.get("GRADE_POINTS", 0) or 0,
                row.get("CREDIT_POINTS", 0) or 0
            ])

        if len(data) == 1:
            continue

        total_credits = sem_data["CREDIT"].sum()
        total_credit_points = sem_data["CREDIT_POINTS"].sum()
        sgpa = round(total_credit_points / total_credits, 2) if total_credits else 0

        data.append([
            Paragraph("Total", bold_subject_style),
            Paragraph(f"SGPA : {sgpa}", bold_subject_style),
            total_credits, "", "", total_credit_points
        ])

        table = Table(data, colWidths=[65, 220, 45, 45, 70, 65], repeatRows=1)
        table.setStyle(TableStyle([
            ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
            ("FONTNAME", (0,0), (-1,-1), "Times-Roman"),
            ("FONTSIZE", (0,0), (-1,-1), 9),
            ("FONTNAME", (0,0), (-1,0), "Times-Bold"),
            ("FONTNAME", (0,-1), (-1,-1), "Times-Bold"),
            ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
            ("ALIGN", (2,1), (-1,-1), "CENTER"),
        ]))

        elements.append(Paragraph(f"<b>Semester {sem}</b>", left_sem_style))
        elements.append(Spacer(1, 6))
        elements.append(table)
        elements.append(Spacer(1, 12))

    # --- Summary ---
    try:
        cgpa = round(student_data["CGPA"].iloc[-1], 2)
    except Exception:
        cgpa = 0
    percentage = round(cgpa * 10, 2)
    total_credit_points = int(student_data["CREDIT_POINTS"].sum())
    total_credit = int(student_data["CREDIT"].sum())
    result = student_data.iloc[0].get("RESULT", "")

    summary_data = [
        ["RESULT", f": {result}", "", "Grand Total Credit Points", f": {total_credit_points}"],
        ["CGPA", f": {cgpa}", "", "Grand Total Credit", f": {total_credit}"],
        ["Percentage", f": {percentage}", "", "", ""]
    ]

    colWidths = [0.12 * (width - doc.leftMargin - doc.rightMargin),
                 0.12 * (width - doc.leftMargin - doc.rightMargin),
                 0.30 * (width - doc.leftMargin - doc.rightMargin),
                 0.28 * (width - doc.leftMargin - doc.rightMargin),
                 0.28 * (width - doc.leftMargin - doc.rightMargin)]

    summary_table = Table(summary_data, colWidths=colWidths, hAlign='LEFT')
    summary_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (1, 0), (1, -1), 'Helvetica-Bold'),
        ('FONTNAME', (4, 0), (4, -1), 'Helvetica-Bold'),
    ]))

    elements.append(summary_table)
    elements.append(Spacer(1, 20))

    # --- Build (no page numbering here) ---
    elements = [x for x in elements if x not in [None, "", []]]
    try:
        doc.build(elements)  # 🚨 no canvasmaker, so no page numbers
        print(f"✅ One-page PDF generated: {filename}")
    except Exception as e:
        print(f"❌ PDF build failed: {e}")


### 5.1 Selection - One page or Multiple pages

In [53]:
def generate_pdf_auto(student_id, student_data, report_date):
    """
    Automatically choose one-page or multipage PDF generation
    depending on the number of rows.
    """

    # Heuristic: if total subject rows are small, it fits in one page
    total_subjects = student_data[["SUB_CODE", "SUB_NAME"]].notna().sum().max()
    
    # Adjust threshold as per your layout (typically ~25–30 rows fits one A4 page)
    if total_subjects <= 15:
        #print(f"ℹ️ Using ONE-PAGE layout for {student_data.iloc[0]['CNAME']}")
        return generate_pdf_onepage(student_id, student_data, report_date)
    else:
        #print(f"ℹ️ Using MULTI-PAGE layout for {student_data.iloc[0]['CNAME']}")
        return generate_pdf(student_id, student_data, report_date)

### 5.2 Main Generator

In [54]:
# Group and generate PDFs, assigning serial numbers
report_date="27-09-2025"
for (student_id, student_data) in df_processed.groupby("RROLL"):
    generate_pdf_auto(student_id, student_data, report_date )

ℹ️ Using MULTI-PAGE layout for PRITY RANI DEKA
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_PRITY RANI DEKA.pdf
ℹ️ Using MULTI-PAGE layout for SUNIT DEY
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_SUNIT DEY.pdf
ℹ️ Using MULTI-PAGE layout for HIRAKJYOTI KALITA
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_HIRAKJYOTI KALITA.pdf
ℹ️ Using MULTI-PAGE layout for NINAD BHATTACHARYYA
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_NINAD BHATTACHARYYA.pdf
ℹ️ Using MULTI-PAGE layout for GARIMA DAS
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_GARIMA DAS.pdf
ℹ️ Using MULTI-PAGE layout for BANASMITA KAKATI
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_BANASMITA KAKATI.pdf
ℹ️ Using MULTI-PAGE layout for JINI AHMED
✅ PDF generated: D:\data\gcu\erp\transcript\transcripts_pdf\Transcript_JINI AHMED.pdf
ℹ️ Using MULTI-PAGE layout for JUBIN NATH
✅ PDF gene

In [ ]:
# Output is generated at file:///D:/Notebooks/Web%20Development/transcripts_pdf/Transcript_BANASMITA%20KAKATI.pdf

### References:

1. https://www.youtube.com/watch?v=q70xzDG6nls&list=PLjNQtX45f0dR9K2sMJ5ad9wVjqslNBIC0
2. https://github.com/bvalgard/create-pdf-with-python-fpdf2
3. https://www.youtube.com/watch?v=p1_7t2SOCIs
4. https://github.com/liannewriting/YouTube-videos-public/blob/main/generate-reports-with-python-sp500/generate_reports_with_python.ipynb
5. https://pyfpdf.readthedocs.io/en/latest/
6. https://github.com/reingart/pyfpdf/blob/master/docs/Tutorial.md